In [130]:
import numpy as np, pandas as pd, math
import matplotlib.pyplot as plt
import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from scipy.io import loadmat

from keras.models import Sequential
#from keras.layers import Dense, LSTM, Recurrent, Dropout, Conv1D, Embedding
from keras.layers import Dense, LSTM, Dropout, Conv1D, Embedding

#### Hyper Parameters
np.random.seed(0)  # Random Seed
N_time_steps = 10  # Time steps of the RNN network
units_list = [20, 50]  # In each layer, for every unit use how many hidden neurons
batch_size = 4  # Each time use how many samples
epochs = 20  # How many epochs to train the model
path = 'irs-tap-traces/'  # Folder that source data is saved
X=  loadmat(path+'irs-1.mat')['abs_irs']
y= loadmat(path+'tap-1.mat')['gt_tap']


#### Normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X.T)  # Use X.Transpose to normalize among each signals
X = X.T
scaler1 = MinMaxScaler(feature_range=(0, 1))
y = scaler1.fit_transform(y)
print(type(X), X.shape, y.shape)

#### Reshape 2-D X (samples, signals) into 3-D (samples * time-steps * window-size)
total_time = X.shape[0]
print('total time: %d ' % total_time)
start_idx = 1
end_idx = X.shape[0] - N_time_steps + 1
X_3d = np.zeros((end_idx, N_time_steps, X.shape[1]))   #initial
y_3d = np.zeros((end_idx, 1))
for i in range(end_idx):
    X_3d[i, :, :] = X[i:i+N_time_steps, :]
    y_3d[i] = y[i + 9]
print(X_3d.shape)
print(y_3d.shape)
print('data process done...')


train_size = int(len(X_3d) * 0.67)
test_size = len(X_3d) - train_size
X_train, X_test = X_3d[:train_size, :], X_3d[train_size:len(X), :]
y_train, y_test = y_3d[:train_size, :], y_3d[train_size:len(X), :]
print(X_train.shape)
print(y_train.shape)

<class 'numpy.ndarray'> (240, 3120) (3120, 1)
total time: 240 
(231, 10, 3120)
(231, 1)
data process done...
(154, 10, 3120)
(154, 1)


In [131]:
#N_time_steps=5
#ll=11
#outll=1
#X_train, X_test = X_3d[:2, 0:N_time_steps, 0:ll], X_3d[2:3, 0:N_time_steps, 0:ll]
#y_train, y_test = y_3d[:2, 0:1], y_3d[2:3, 0:1]
#print(X_train.shape)
#print(y_train.shape)

units_lists=[20,50]
model = Sequential()
# model.add(Conv1D(128, kernel_size=1, activation='relu', input_shape=(N_time_steps, X.shape[1])))
model.add(LSTM(input_shape=(N_time_steps, X.shape[1]), units=units_list[0], return_sequences=True))  # First Layer
model.add(Dropout(0.2))
model.add(LSTM(input_shape=(N_time_steps, X.shape[1]), units=units_list[1], return_sequences=False))  # Second Layer
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()
print(model.get_config())


model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1);
#model.fit(X_train, y_train, epochs=epochs, verbose=0);

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 10, 20)            251280    
_________________________________________________________________
dropout_10 (Dropout)         (None, 10, 20)            0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 50)                14200     
_________________________________________________________________
dropout_11 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 265,531
Trainable params: 265,531
Non-trainable params: 0
_________________________________________________________________
{'name': 'sequential_5', 'layers': [{'class_name': 'LSTM', 'config': {'name': 'lstm_8', 'trainable': True, 'batch_

In [ ]:
weights = model.get_weights()
for i in range(len(weights)):
    print(weights[i].shape)
npweights = np.array(weights)

for i in range(len(weights)):    
    if( len(npweights[i].shape) == 2):
        weights[i] = weights[i][0:2, 0:4]
    if( len(npweights[i].shape) == 1):
        weights[i] = weights[i][0:2]

print(weights)

weights = np.array(weights)


for i in range(len(weights)):
    print(weights[i].shape)
for i in range(len(weights)):
    if( len(weights[i].shape) == 2):
        weights[i] = weights[i][0:2, 0:4]
    if( len(weights[i].shape) == 1):
        weights[i] = weights[i][0:2]

        
print(weights)

In [144]:
weights = np.array(model.get_weights())
#for i in range(2):
    #    print(weights[0][i][0:3])
    #    print(weights[1][i][0:3])
    #    print(weights[2][i])
#for i in range(len(weights)):
   # print(weights[i].shape)
#print(weights[0][0][0:3])

flatweights=np.array([])
for i in range(8):    
    weights[i] = weights[i].flatten();
    flatweights = np.hstack((flatweights, weights[i]))



print(flatweights.shape)
print(flatweights[251280:251280+4], flatweights[251280 + 200:251280+200+4])
print(flatweights[265480:265480+4], flatweights[265480 + 49], flatweights[265480 + 50])
#for i in range(len(weights)):
#    np.savetxt('weights%d.txt' % i, weights[i], newline='\n')
np.savetxt('flatweights.txt', flatweights, fmt="%e")
xtestp=X_test[0:2]
xtestflat = xtestp.flatten()
np.savetxt('flattest.txt', xtestflat, fmt="%e")
print ('---')
print(xtestflat[0:2],xtestflat[3120:3122], xtestflat[10*3120:31200+2])
print(xtestp[0:2, 0:3])
print("--test.shape:")
print(X_test.shape)
prey=model.predict(X_test)
print(np.array(prey).shape)

(265531,)
[ 0.026374  0.137397 -0.123673 -0.039094] [ 0.079275  0.101554  0.04169  -0.159007]
[-0.096018 -0.01741   0.150495  0.090195] 0.28609195351600647 0.012956217862665653
---
[0.00557  0.005034] [0.00606  0.005377] [0.00606  0.005377]
[[[0.00557  0.005034 0.004171 ... 0.001261 0.002294 0.002928]
  [0.00606  0.005377 0.004575 ... 0.001345 0.002522 0.003226]
  [0.006536 0.005758 0.005044 ... 0.001315 0.002644 0.003383]]

 [[0.00606  0.005377 0.004575 ... 0.001345 0.002522 0.003226]
  [0.006536 0.005758 0.005044 ... 0.001315 0.002644 0.003383]
  [0.007027 0.006216 0.005586 ... 0.001199 0.002703 0.003411]]]
--test.shape:
(2, 10, 3120)
(2, 1)


In [139]:
def sigmoid(x):
    return(1.0/(1.0+np.exp(-x)))

def LSTMlayer(warr,uarr, barr,x_t,h_tm1,c_tm1):


    s_t = (x_t.dot(warr) + h_tm1.dot(uarr) + barr)
    #print(x_t[0, 0:5], warr[0, 0:5], h_tm1[0, 0:5], uarr[0, 0:5], barr[0:5])
    #print(s_t[0, 0:10])
    hunit = uarr.shape[0]
    i  = sigmoid(s_t[:,:hunit])
    f  = sigmoid(s_t[:,1*hunit:2*hunit])
    _c = np.tanh(s_t[:,2*hunit:3*hunit])
    o  = sigmoid(s_t[:,3*hunit:])
    #print(i[0,0:5], f[0,0:5], _c[0,0:5], o[0,0:5]);
    c_t = i*_c + f*c_tm1
    h_t = o*np.tanh(c_t)
    #print(h_t[0, 0:5])
    #print("")
    #print("")
    return(h_t,c_t)


def DenseLayer(x, w, b):
    return x.dot(w)+b;

# n: window size
# hunits: output dim
# inputlen: input dim
def intermediate_out(n,warr,uarr,barr, X_test, hunits, inputlen):
    for i in range(0, n+1):
        if i==0:
            c_tm1 = np.array([0]*hunits, dtype=np.float32).reshape(1,hunits)
            h_tm1 = np.array([0]*hunits, dtype=np.float32).reshape(1,hunits)
            h_t,ct = LSTMlayer(warr,uarr, barr,X_test[0:1, 0:inputlen],h_tm1,c_tm1)
        else:
            h_t,ct = LSTMlayer(warr,uarr, barr,X_test[i:i+1, 0:inputlen],h_t,ct)
    return h_t

In [140]:
tmp = X_test
X_test=X_test[0:2]
hunits=20
windowsize=10

weightLSTM = np.array(model.get_weights()[0:3])
warr,uarr, barr = weightLSTM

inputlen = X_test.shape[2]
samplenum = X_test.shape[0]
print("inputlen1")
print(inputlen)
xlstm1 = np.zeros((X_test.shape[0], 10, hunits))
print(X_test.shape)

for n in range(samplenum):
    for i in range(windowsize):
        xlstm1[n][i] = intermediate_out(i,warr,uarr,barr,X_test[n], hunits, inputlen)

output1len = hunits

    
## layer lstm 2

print(prey.shape)
print(xlstm1.shape)
print(prey[0:2])
print(xlstm1[1, 0:2, :])

inputlen1
3120
(2, 10, 3120)
(77, 1)
(2, 10, 20)
[[0.038122]
 [0.035279]]
[[ 5.188636e-03 -5.976483e-04 -2.311663e-03  8.572704e-03  5.661738e-03
  -9.532309e-03  2.895232e-03  4.375157e-03  2.059858e-03 -1.228063e-04
  -5.630913e-03 -2.585759e-03  3.775928e-03  4.204208e-03  4.335599e-03
   3.888438e-03  1.922139e-03 -5.025594e-03 -5.130682e-03 -4.510193e-03]
 [ 8.289863e-03 -1.463899e-03 -2.871907e-03  1.466659e-02  1.034203e-02
  -1.538972e-02  4.517830e-03  7.927158e-03  2.809095e-03  7.943323e-05
  -9.658235e-03 -4.104369e-03  5.577469e-03  7.018458e-03  6.357080e-03
   5.708643e-03  4.082741e-03 -9.161425e-03 -8.458011e-03 -7.608317e-03]]


In [129]:
hunits = 50
windowsize = 10

weightLSTM2 = np.array(model.get_weights()[3:6])
warr,uarr, barr = weightLSTM2

print(weightLSTM2[0][0:2, 0:4])
inputlen = xlstm1.shape[2]
samplenum = xlstm1.shape[0]
xlstm2 = np.zeros((xlstm1.shape[0], hunits))

print("inputlen2, samplenum")
print(inputlen ,samplenum, xlstm2.shape)


for n in range(samplenum):
    xlstm2[n] = intermediate_out(i,warr,uarr,barr, xlstm1[n], hunits, inputlen)
    #print(a.shape)
    #xlstm2[n][0] = a
    
print(xlstm2[0, 0:10])
print(prey[0, 0:10])
print(xlstm2[1, 0:10])
print(prey[1, 0:10])


weightDENSE = np.array(model.get_weights()[6:8])
warr, barr = weightDENSE

xdense = np.zeros((xlstm2.shape[0], 1))
for n in range(xlstm2.shape[0]):
    xdense[n]=DenseLayer(xlstm2[n].T, warr, barr);

    
    
np.set_printoptions(precision=6)
test=10
for i in range(test):
    print(xdense[i])
    print(prey[i])

[[-0.010914 -0.091002 -0.075239  0.127657]
 [-0.031738 -0.087789  0.127712  0.125736]]
inputlen2, samplenum
20 2 (2, 50)
[ 8.534716e-05  2.616592e-03 -4.559041e-03 -4.810492e-03 -9.369729e-03] [-0.010914 -0.091002 -0.075239  0.127657  0.008229] [0. 0. 0. 0. 0.] [0.040828 0.075186 0.138159 0.076118 0.085629] [-0.029422 -0.001511 -0.001138 -0.024965 -0.013731]
[-0.029137 -0.001793  0.000263 -0.027276 -0.014286 -0.01623  -0.014642
 -0.026064 -0.029986 -0.01028 ]
[0.492716 0.499552 0.500066 0.493181 0.496428] [0.72641  0.731268 0.73071  0.72606  0.730487] [0.014616 0.018992 0.02938  0.011996 0.018104] [0.492199 0.50006  0.500234 0.493874 0.496534]
[0.003544 0.004744 0.007349 0.002922 0.004462]


[-7.627251e-05  3.731392e-03 -7.745006e-03 -8.995222e-03 -1.641552e-02] [-0.010914 -0.091002 -0.075239  0.127657  0.008229] [0.003544 0.004744 0.007349 0.002922 0.004462] [0.040828 0.075186 0.138159 0.076118 0.085629] [-0.029422 -0.001511 -0.001138 -0.024965 -0.013731]
[-0.02751  -0.003116  0.00260

ValueError: not enough values to unpack (expected 2, got 0)

In [42]:
#xs  = np.array([0.003,0.002,1])
for i in range(len(xs)):
    x_t = xs[i].reshape(1,1)
    h_tm1,c_tm1 = LSTMlayer(weights,x_t,h_tm1,c_tm1)
print("h3={}".format(h_tm1))
print("c3={}".format(c_tm1))
batch_size = 1
len_ts = len(xs)
nfeature = X_test.shape[2]

inp = layers.Input(batch_shape= (batch_size, len_ts, nfeature),
                       name="input")  
rnn,s,c = layers.LSTM(hunits, 
                         return_sequences=True,
                         stateful=False,
                         return_state=True,
                         name="RNN")(inp)
states = models.Model(inputs=[inp],outputs=[s,c, rnn])

for layer in states.layers:
    for layer1 in model1.layers:
        if layer.name == layer1.name:
            layer.set_weights(layer1.get_weights())
            
h_t_keras, c_t_keras, rnn = states.predict(xs.reshape(1,len_ts,1))
print("h3={}".format(h_t_keras))
print("c3={}".format(c_t_keras))
def get_LSTM_UWb(weight):
    '''
    weight must be output of LSTM's layer.get_weights()
    W: weights for input
    U: weights for hidden states
    b: bias
    '''
    warr,uarr, barr = weight
    gates = ["i","f","c","o"]
    hunit = uarr.shape[0]
    U, W, b = {},{},{}
    for i1,i2 in enumerate(range(0,len(barr),hunit)):
        
        W[gates[i1]] = warr[:,i2:i2+hunit]
        U[gates[i1]] = uarr[:,i2:i2+hunit]
        b[gates[i1]] = barr[i2:i2+hunit].reshape(hunit,1)
    return(W,U,b)

def get_LSTMweights(model1):
    for layer in model1.layers:
        if "LSTM" in str(layer):
            w = layer.get_weights()
            W,U,b = get_LSTM_UWb(w)
            break
    return W,U,b
def vectorize_with_labels(W,U,b):
    bs,bs_label,ws,ws_label,us,us_label=[],[],[],[],[],[]
    for k in ["i","f","c","o"]:
        temp = list(W[k].flatten())
        ws_label.extend(["W_"+k]*len(temp))
        ws.extend(temp)

        temp = list(U[k].flatten())
        us_label.extend(["U_"+k]*len(temp))
        us.extend(temp)    

        temp = list(b[k].flatten())
        bs_label.extend(["b_"+k]*len(temp))
        bs.extend(temp)  
    weight = ws + us + bs
    wlabel = ws_label + us_label + bs_label
    return(weight,wlabel)

NameError: name 'xs' is not defined